#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 한글폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumGothic')
sns.set(font="NanumGothic",#"NanumGothicCoding",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

# 경고 표시 무시
import warnings
warnings.filterwarnings('ignore')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [52]:
# 구글 코랩 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.
      - test 데이터는 조금있다 불러올게요...


In [53]:
data =  pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2/2023.10.25_미니프로젝트5차_실습자료밀 데이터/데이터/data01_train.csv')

In [54]:
from joblib import load

In [55]:
# 탐색적 데이터 분석에서 관점 1 ~ 관점 8까지 상위 5개 feautre를 추출해서 나온 feature
vv = load('/content/drive/MyDrive/미니프로젝트5차part2/2023.10.25_미니프로젝트5차_실습자료밀 데이터/데이터/variables.pkl')

In [56]:
vv

['tBodyAcc-energy()-X',
 'angle(X,gravityMean)',
 'tBodyGyroJerk-iqr()-Z',
 'tGravityAcc-arCoeff()-Z,3',
 'tGravityAcc-min()-X',
 'tGravityAcc-max()-Y',
 'tBodyAccMag-std()',
 'tBodyGyroJerk-mad()-X',
 'fBodyAccMag-sma()',
 'tGravityAcc-mean()-Z',
 'tGravityAcc-min()-Y',
 'fBodyAcc-mad()-X',
 'tGravityAcc-mean()-Y',
 'angle(Y,gravityMean)',
 'fBodyAccMag-std()',
 'tGravityAccMag-std()',
 'tGravityAcc-mean()-X',
 'tBodyAccJerk-mad()-X',
 'fBodyAcc-energy()-X',
 'tBodyAcc-max()-X',
 'tBodyAccJerk-mad()-Y',
 'tGravityAcc-arCoeff()-X,1',
 'tGravityAcc-energy()-X',
 'tGravityAcc-max()-X',
 'fBodyAcc-meanFreq()-Z',
 'tBodyAccJerk-iqr()-Y',
 'tGravityAcc-arCoeff()-Z,2']

* 변수가 12, 14개일 떄 HistGradientBoosting 알고리즘이 최적의 성능을 내서 이를 채택한다.
  * 변수는 12개를 사용할 예정이며
  * 알고리즘은 HistGradientBoosting을 사용할 예정이다.


In [57]:
data2 = data[vv[0:12:1] + ['Activity']]

In [58]:
data2.shape

(5881, 13)

In [59]:
data2.head()

,tBodyAcc-energy()-X,"angle(X,gravityMean)",tBodyGyroJerk-iqr()-Z,"tGravityAcc-arCoeff()-Z,3",tGravityAcc-min()-X,tGravityAcc-max()-Y,tBodyAccMag-std(),tBodyGyroJerk-mad()-X,fBodyAccMag-sma(),tGravityAcc-mean()-Z,tGravityAcc-min()-Y,fBodyAcc-mad()-X,Activity
0,-0.999869,-0.601120,-0.986843,-0.385893,0.891969,-0.370494,-0.981876,-0.986107,-0.985206,-0.205765,-0.334921,-0.988021,STANDING
1,-0.999873,0.345205,-0.992360,0.017077,-0.095343,0.873312,-0.990870,-0.992184,-0.989450,0.234498,0.937432,-0.988538,LAYING
2,-0.999988,-0.833564,-0.990670,0.015440,0.985606,-0.195671,-0.992562,-0.984787,-0.990805,0.078530,-0.145801,-0.997419,STANDING
3,-0.703590,-0.695819,-0.430966,-0.430046,0.943954,-0.309430,-0.459911,-0.363761,-0.407268,-0.129686,-0.275699,-0.146359,WALKING
4,-0.404271,-0.705029,-0.699558,-0.790986,0.901943,-0.236084,0.152179,-0.554993,0.191831,-0.163806,-0.263482,0.105888,WALKING_DOWNSTAIRS


## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [60]:
data2['Activity'].unique()

array(['STANDING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS', 'SITTING'], dtype=object)

In [61]:
# 'Activity_dynamic' 변수 추가
# '정적 행동(Laying, Sitting, Standing)'이면 0을 부여
# '동적 행동(Walking, Walking-Up, Walking-Down)'이면 1을 부여
data2['Activity_dynamic'] = data2['Activity'].apply(lambda x : 1 if x in ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'] else 0)

In [62]:
data2['Activity_dynamic'].value_counts() # 이정도면 클래스 불균형이 있지는 않다...

0    3234
1    2647
Name: Activity_dynamic, dtype: int64

* 이정도면 0과 1에 데이터 불균형이 있다고 판단하지 않겠다.

* 'Activity' 변수에 있는 값들을 숫자 형태로 변환한다.
   * 일부 머신러닝 알고리즘은 문자열 형태도 받아들이지만, 전체적으로는 숫자 형태로 변환하는 것이 마음 편하다.

In [63]:
# 'Activity' 변수에 있는 값들을 숫자 형태로 변경한다.
data2['Activity'] = data2['Activity'].map({
    'STANDING' : 0,
    'LAYING' : 1,
    'SITTING' : 2,
    'WALKING' : 3,
    'WALKING_DOWNSTAIRS' : 4,
    'WALKING_UPSTAIRS' : 5,
})

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
# 입력 변수 X와 타겟 변수 y1, y2 분리
X = data2.drop(columns=['Activity', 'Activity_dynamic'], axis=1)
y1 = data2['Activity']
y2 = data2['Activity_dynamic']

# 훈련 세트와 검증 세트 분할 (train : val = 8 : 2)
X_train, X_val, y1_train, y1_val, y2_train, y2_val = train_test_split(X, y1, y2, test_size=0.2, random_state=42)

* 모두 index를 0부터 시작하는 연속적인 정수로 부여한다.
  * reset_index()를 이용한다.

In [66]:
X_train.reset_index(drop=True, inplace=True)
X_val.reset_index(drop=True, inplace=True)
y1_train.reset_index(drop=True, inplace=True)
y1_val.reset_index(drop=True, inplace=True)
y2_train.reset_index(drop=True, inplace=True)
y2_val.reset_index(drop=True, inplace=True)

* 표준화를 진행한다.
  * StandardScaler를 사용한다.

In [67]:
# 표준화 진행
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train_scaled = pd.DataFrame(ss.fit_transform(X_train), columns=X_train.columns) # 2차원 넘파이 배열 후 데이터프레임으로 전환
X_val_scaled = pd.DataFrame(ss.transform(X_val), columns=X_val.columns) # 2차원 넘파이 배열 후 데이터프레임으로 전환

In [68]:
X_train_scaled.shape, X_val_scaled.shape

((4704, 12), (1177, 12))

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 :

In [69]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [70]:
# 정적 행동(0)과 동적 행동(1)을 구분하는 모델
model1 = HistGradientBoostingClassifier(random_state=42, )
model1.fit(X_train_scaled, y2_train)

HistGradientBoostingClassifier(random_state=42)

In [71]:
model1_predict= model1.predict(X_val_scaled) # 예측값이 0이 나왔는지 1이 나왔는지 확인

In [72]:
model1_predict # 예측값은 0(정적 행동), 1(동적 행동)로 구성된다.

array([0, 1, 0, ..., 0, 1, 0])

#### 2) 알고리즘2 :

* Train set 전체에서 Laying(0), Sitting(1),
Standing(2) 데이터만 뽑아서  3 - Class 분류하는 모델 생성할지

* Train set 전체에서 Walking(3), W-Up(4),
W-Down(5) 데이터만 뽑아서 3 - Class 분류하는 모델 생성할지 출발점이 되는 곳이다.

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying(0), Sitting(1), Standing(2))인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [73]:
static_index = y1_train[y1_train.isin([0, 1, 2])].index # y1_train에서 0, 1, 2만 있는 index를 추출한다.
static_X_train_scaled = X_train_scaled.loc[static_index] # 해당된 index에만 적용
static_y_train = y1_train.loc[static_index] # 해당된 index에만 적용

In [74]:
static_X_train_scaled.shape, static_y_train.shape

((2577, 12), (2577,))

In [75]:
static_y_train.value_counts()

1    884
0    861
2    832
Name: Activity, dtype: int64

* 0, 1, 2에 대한 데이터 불균형은 없는 것으로 판단된다.

* 정적 행동을 분류하는 모델 2-1을 만든다.

In [76]:
# 정적 행동을 분류하는(0, 1, 2) 모델 2-1
model2_1 = HistGradientBoostingClassifier(random_state=42)
model2_1.fit(static_X_train_scaled, static_y_train)

HistGradientBoostingClassifier(random_state=42)

In [77]:
# 모델 2-1 Laying(0), Sitting(1), Standing(2)를 다중 분류하는 함수
def static_behavior_classification(idx):
    result = model2_1.predict(X_val_scaled.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [78]:
dynamic_index = y1_train[y1_train.isin([3, 4, 5])].index # y1_train에서 3, 4, 5만 있는 index를 추출한다.
dynamic_X_train_scaled = X_train_scaled.loc[dynamic_index] # 해당된 index에만 적용
dynamic_y_train = y1_train.loc[dynamic_index] # 해당된 index에만 적용

In [79]:
dynamic_X_train_scaled.shape, dynamic_y_train.shape

((2127, 12), (2127,))

In [80]:
dynamic_y_train.value_counts()

3    800
5    681
4    646
Name: Activity, dtype: int64

* 이정도면 3, 4, 5에 대한 데이터 불균형은 없다고 봐도 무방하다.

* 동적 행동(3, 4, 5)을 분류하는 모델 2-2를 만든다.

In [81]:
# 동적 행동을 분류하는(3, 4, 5) 모델 2-2
model2_2 = HistGradientBoostingClassifier(random_state=42)
model2_2.fit(dynamic_X_train_scaled, dynamic_y_train)

HistGradientBoostingClassifier(random_state=42)

In [82]:
# 모델 2-2 Walking(3), Walking-Up(4), Walking-Down(5)를 다중 분류하는 함수
def dynamic_behavior_classification(idx):
    result = model2_2.predict(X_val_scaled.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

* 모델 2-1, 2-2를 실행한다.

In [83]:
model2_predict = [] # model2_predict(예측값)는 y1_val(실제값)과 비교할 예정이다.

for idx, mp in enumerate(model1_predict): # model1_predict(예측값)을 이용해서 모델 2(모델 2-1, 모델 2-2)를 수행할 예정이다.
    if mp == 0:
        result = static_behavior_classification(idx)
        model2_predict.append(result)
    else: # mp == 1
        result = dynamic_behavior_classification(idx)
        model2_predict.append(result)

model2_predict = np.array(model2_predict) # 1차원 넘파이 배열로 변환

* 예측값과 실제값 비교

In [84]:
from sklearn.metrics import accuracy_score

In [85]:
print('정확도 : ', accuracy_score(y1_val.values, model2_predict)) # 실제값과 예측값을 1차 넘파이 배열 형태로 변환해서 accuracy_score를 진행한다.

정확도 :  0.9864061172472387


* 아직까지는 test 데이터로는 정확도를 내지는 않았어요, X_val를 테스트로
삼아 진행했었습니다.



### data_01_test.csv 파일을 가져와서 전처리한후 테스트 데이터로 사용한다.

In [86]:
test_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2/2023.10.25_미니프로젝트5차_실습자료밀 데이터/데이터/data01_test.csv')

* 변수는 12개만 사용하며
  * 알고리즘은 HistGradientBoosting을 사용한다.

In [87]:
test_data = test_data[vv[0:12:1] + ['Activity']]

In [88]:
test_data.shape

(1471, 13)

* 'Activity' 변수에 있는 값들은 숫자 형태로 변경한다,
  * 일부 머신러닝 알고리즘은 문자열 형태를 지원하여 자동적으로 숫자 형태로 변경하지만<br> 전체적으로는 아니여서 그냥 숫자 형태로 변경하는 것이 마음이 편하다.

In [89]:
# 'Activity' 변수에 있는 값들을 숫자 형태로 변경한다.
test_data['Activity'] = test_data['Activity'].map({
    'STANDING' : 0,
    'LAYING' : 1,
    'SITTING' : 2,
    'WALKING' : 3,
    'WALKING_DOWNSTAIRS' : 4,
    'WALKING_UPSTAIRS' : 5,
})

In [90]:
# 실제값을 저장한다. -> 1차원 넘파이 배열
y_true = test_data['Activity'].values

In [91]:
# 표준화 진행 -> 2. 데이터 전처리 부분에서의 표준화 처리하는데에서 참고했다.
test_data = test_data.drop(['Activity'], axis=1)
test_data_scaled = pd.DataFrame(ss.transform(test_data), columns=test_data.columns)

* 정적 행동(0)과 동적 행동(1)를 예측하는 model1를 이용한다.

In [92]:
# 단계 1에 있는 정적 행동(0)과 동적 행동(1)을 예측하는 model1를 이용한다. 그 결과 정적 행동(0)과 동적 행동(1)을 얻을 수 있다.
model1_predict = model1.predict(test_data_scaled)

In [93]:
model1_predict

array([0, 0, 1, ..., 1, 0, 1])

In [94]:
# 모델 2-1 Laying(0), Sitting(1), Standing(2)를 다중 분류하는 함수
def static_behavior_classification(idx):
    result = model2_1.predict(test_data_scaled.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

In [95]:
# 모델 2-2 Walking(3), Walking-Up(4), Walking-Down(5)를 다중 분류하는 함수
def dynamic_behavior_classification(idx):
    # print(model2_2.predict_proba(X_val_scaled.iloc[idx].values.reshape(1, -1)))
    result = model2_2.predict(test_data_scaled.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

* 모델 2-1, 모델 2-2를 실행한다.

In [96]:
model2_predict = [] # model2_predict(예측값)는 y_true(실제값)과 비교할 예정이다.

for idx, mp in enumerate(model1_predict): # model_predict(예측값)을 이용해서 모델 2(모델 2-1, 모델 2-2)를 수행할 예정이다.
    if mp == 0:
        result = static_behavior_classification(idx)
        model2_predict.append(result)
    else: # mp == 1
        result = dynamic_behavior_classification(idx)
        model2_predict.append(result)

model2_predict = np.array(model2_predict) # 1차원 넘파이 배열로 변환

* 실제값과 예측값을 비교한다.

In [97]:
print('정확도 : ', accuracy_score(y_true, model2_predict)) # 실제값과 예측값을 1차 넘파이 배열 형태로 변환해서 accuracy_score를 진행한다.

정확도 :  0.9775662814411965


### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

* 테스트 데이터(test)가 들어왔을 떄 함수로 만든다.

In [98]:
# 모델 2-1 Laying(0), Sitting(1), Standing(2)를 다중 분류하는 함수
def static_behavior_classification(idx):
    result = model2_1.predict(test_data_scaled.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

# 모델 2-2 Walking(3), Walking-Up(4), Walking-Down(5)를 다중 분류하는 함수
def dynamic_behavior_classification(idx):
    result = model2_2.predict(test_data_scaled.iloc[idx].values.reshape(1, -1)) # Series를 2차원 넘파이 배열로 왜냐하면 머신러닝, 딥러닝 입력 형태가 데이터프레임이나 2차원 넘파이 배열이기 떄문
    return result[0]

def function(test_data):
    # 1. 변수는 27개만 사용하며 알고리즘은 HistGradientBoosting을 사용할 예정이다.
    test_data = test_data[vv[0:12:1] + ['Activity']]

    # 2. 'Activity' 변수에 있는 값들을 숫자 형태로 변경한다.
    test_data['Activity'] = test_data['Activity'].map({
        'STANDING' : 0,
        'LAYING' : 1,
        'SITTING' : 2,
        'WALKING' : 3,
        'WALKING_DOWNSTAIRS' : 4,
        'WALKING_UPSTAIRS' : 5,
    })

    # 3. 실제값을 저장한다. -> 1차원 넘파이 배열
    y_true = test_data['Activity'].values

    # 4. 표준화 진행 -> 2. 데이터 전처리 부분에서의 표준화 처리하는데에서 참고했다.
    test_data = test_data.drop(['Activity'], axis=1)
    test_data_scaled = pd.DataFrame(ss.transform(test_data), columns=test_data.columns)

    # 5. 단계 1에 있는 정적 행동(0)과 동적 행동(1)을 예측하는 model1를 이용한다. 그 결과 정적 행동(0)과 동적 행동(1)을 얻을 수 있다.
    model1_predict = model1.predict(test_data_scaled)

    # 6. 모델 2-1, 2-2를 수행한다.
    model2_predict = [] # model2_predict(예측값)는 y_true(실제값)과 비교할 예정이다.
    for idx, mp in enumerate(model1_predict): # model_predict(예측값)을 이용해서 모델 2(모델 2-1, 모델 2-2)를 수행할 예정이다.
        if mp == 0:
            result = static_behavior_classification(idx)
            model2_predict.append(result)
        else: # mp == 1
            result = dynamic_behavior_classification(idx)
            model2_predict.append(result)

    model2_predict = np.array(model2_predict) # 1차원 넘파이 배열로 변환

    # 7. 정확도 판단
    print('정확도 : ', accuracy_score(y_true, model2_predict)) # 실제값과 예측값을 1차 넘파이 배열 형태로 변환해서 accuracy_score를 진행한다.

In [99]:
test_data = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차part2/2023.10.25_미니프로젝트5차_실습자료밀 데이터/데이터/data01_test.csv')
function(test_data)

정확도 :  0.9775662814411965
